In [ ]:
# Import Libraries
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from skimage import io, measure, feature

In [ ]:
# Access the dataset folder
AllDatasetDirPath = r"D:\AI-Practical-Tasks\2023-2024\final\Group D\MangoLeafBD Dataset"
FolderList = my_list = os.listdir(AllDatasetDirPath)
print(FolderList)

In [ ]:
def create_dataset(dataset_dir):
    names = [
        "area",
        "perimeter",
        "physiological_length",
        "physiological_width",
        "aspect_ratio",
        "rectangularity",
        "circularity",
        "major_axis",
        "minor_axis",
        "convex_area",
        "convex_ratio",
        "mean_r",
        "mean_g",
        "mean_b",
        "stddev_r",
        "stddev_g",
        "stddev_b",
        "contrast",
        "correlation",
        "inverse_difference_moments",
        "entropy",
        "classlabel",
    ]

    all_dataset_results = pd.DataFrame(columns=names)

    for _, dirs, _ in os.walk(dataset_dir):
        if len(dirs) > 0:
            categories = dirs
            break

    for category in categories:
        category_path = os.path.join(dataset_dir, category)
        class_label = categories.index(category)
        for filename in os.listdir(category_path):
            imgpath = os.path.join(category_path, filename)
            main_img = cv2.imread(imgpath)

            # Creating target labels
            classlabel = class_label

            # Preprocessing

            # Convert the image to RGB
            img = cv2.cvtColor(main_img, cv2.COLOR_BGR2RGB)

            # Convert the image to Grayscale
            gray = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)

            # Perform thresholding
            _, thresholded = cv2.threshold(
                gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
            )

            # Find contours
            contours, _ = cv2.findContours(
                thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
            )

            # Get the largest contour (assumed to be the leaf)
            leaf_contour = max(contours, key=cv2.contourArea)

            # Shape based features
            x, y, w, h = cv2.boundingRect(leaf_contour)
            area = cv2.contourArea(leaf_contour)
            perimeter = cv2.arcLength(leaf_contour, True)
            x, y, w, h = cv2.boundingRect(leaf_contour)
            physiological_length = max(w, h)
            physiological_width = min(w, h)
            aspect_ratio = float(physiological_length) / physiological_width
            rectangularity = area / (physiological_length * physiological_width)
            circularity = (4 * np.pi * area) / (perimeter**2)
            convex_area = cv2.contourArea(cv2.convexHull(leaf_contour))
            convex_ratio = area / convex_area

            # Color based features
            b, g, r = cv2.split(img)

            # Calculate mean and standard deviation for each channel
            mean_b = np.mean(b)
            mean_g = np.mean(g)
            mean_r = np.mean(r)

            std_b = np.std(b)
            std_g = np.std(g)
            std_r = np.std(r)

            # Texture based features
            textures = mt.features.haralick(gray)
            ht_mean = textures.mean(axis=0)
            contrast = ht_mean[1]
            correlation = ht_mean[2]
            inverse_diff_moments = ht_mean[4]
            entropy = ht_mean[8]

            vector = [
                area,
                perimeter,
                physiological_length,
                physiological_width,
                aspect_ratio,
                rectangularity,
                circularity,
                w,
                h,
                convex_area,
                convex_ratio,
                mean_r,
                mean_g,
                mean_b,
                std_r,
                std_g,
                std_b,
                contrast,
                correlation,
                inverse_diff_moments,
                entropy,
                classlabel,
            ]

            # Append the feature vector to the DataFrame
            df_temp = pd.DataFrame([vector], columns=names)
            all_dataset_results = pd.concat(
                [all_dataset_results, df_temp], ignore_index=True
            )

    return all_dataset_results

In [ ]:
dataset = create_dataset(AllDatasetDirPath)

In [ ]:
dataset.to_csv("features.csv")

In [ ]:
dataset.shape

In [ ]:
dataset.head()

In [ ]:
def overlay_features(image_path):
    # Read the image
    main_img = cv2.imread(image_path)
    original_img = main_img.copy()  # Create a copy for visualization

    # Preprocessing
    gray = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(
        gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )
    contours, _ = cv2.findContours(
        thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    leaf_contour = max(contours, key=cv2.contourArea)

    # Extract features
    x, y, w, h = cv2.boundingRect(leaf_contour)
    area = cv2.contourArea(leaf_contour)
    perimeter = cv2.arcLength(leaf_contour, True)
    physiological_length = max(w, h)
    physiological_width = min(w, h)
    aspect_ratio = float(physiological_length) / physiological_width

    # Visualize the image with features overlay
    cv2.rectangle(
        original_img, (x, y), (x + w, y + h), (0, 255, 0), 2
    )  # Draw bounding box

    # Annotate extracted features
    text = (
        f"Area: {area:.2f}, Perimeter: {perimeter:.2f},Aspect Ratio: {aspect_ratio:.2f}"
    )
    cv2.putText(
        original_img,
        text,
        (0, y + 300),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.266,
        (0, 0, 0),
        1,
        cv2.LINE_AA,
    )

    # Display the image with annotations
    plt.figure(figsize=(10, 18))
    plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    plt.title("Image with Extracted Features Overlay")
    plt.axis("off")
    plt.show()

In [ ]:
# Example usage
image_path = r"D:\AI-Practical-Tasks\2023-2024\final\Group D\MangoLeafBD Dataset\Anthracnose\20211008_124249 (Custom).jpg"  # Replace this with the path to your image file
overlay_features(image_path)